In [1]:
using LinearAlgebra
using TensorKit
using JSON
cd("D:\\My Documents\\Code\\Julia_codes\\Tensor network\\IPEPS_TensorKit\\kagome\\SU2_PG")
#push!(LOAD_PATH, "D:\\My Documents\\Code\\Julia_codes\\Tensor network\\IPEPS_TensorKit\\kagome\\SU2_PG")
include("kagome_load_tensor.jl")

include("kagome_model.jl")
include("kagome_IPESS.jl")

D=6
chi=36
tol=1e-10

J1=0.80902;
J2=0;
J3=0;
Jchi=0;
Jtrip=0.5878;


J1=1;
J2=0;
J3=0;
Jchi=0;
Jtrip=0;

CTM_ite_nums=150;
CTM_trun_tol=1e-14;



In [15]:

parameters=Dict([("J1", J1), ("J2", J2), ("J3", J3), ("Jchi", Jchi), ("Jtrip", Jtrip)]);

A_set,B_set,A1_set,A2_set, A_set_occu,B_set_occu,A1_set_occu,A2_set_occu, S_label, Sz_label, virtual_particle, Va, Vb=construct_tensor(D);

filenm="LS_D_"*string(D)*"_chi_40.json"
json_dict=read_json_state(filenm);

bond_tensor,triangle_tensor=construct_su2_PG_IPESS(json_dict,A_set,B_set,A1_set,A2_set, A_set_occu,B_set_occu,A1_set_occu,A2_set_occu, S_label, Sz_label, virtual_particle, Va, Vb);

PEPS_tensor=bond_tensor;
@tensor PEPS_tensor[:] := bond_tensor[-1,1,-5]*bond_tensor[4,3,-6]*bond_tensor[-4,2,-7]*triangle_tensor[1,3,2]*triangle_tensor[4,-2,-3];
A_unfused=PEPS_tensor;

U_phy=unitary(fuse(space(PEPS_tensor, 5) ⊗ space(PEPS_tensor, 6) ⊗ space(PEPS_tensor, 7)), space(PEPS_tensor, 5) ⊗ space(PEPS_tensor, 6) ⊗ space(PEPS_tensor, 7));
@tensor A_fused[:] :=PEPS_tensor[-1,-2,-3,-4,1,2,3]*U_phy[-5,1,2,3];


In [16]:

A_fused=permute(A_fused,(2,3,4,1,5,),())#to compare with python
A_unfused=permute(A_unfused,(2,3,4,1,5,6,7,),())#to compare with python


TensorMap((Rep[SU₂](0=>1, 1/2=>1, 1=>1)' ⊗ Rep[SU₂](0=>1, 1/2=>1, 1=>1)' ⊗ Rep[SU₂](0=>1, 1/2=>1, 1=>1) ⊗ Rep[SU₂](0=>1, 1/2=>1, 1=>1) ⊗ Rep[SU₂](1/2=>1)' ⊗ Rep[SU₂](1/2=>1)' ⊗ Rep[SU₂](1/2=>1)') ← ProductSpace{GradedSpace{SU2Irrep, TensorKit.SortedVectorDict{SU2Irrep, Int64}}, 0}()):
* Data for fusiontree FusionTree{Irrep[SU₂]}((1, 1, 0, 1/2, 1/2, 1/2, 1/2), 0, (true, true, false, false, true, true, true), (0, 0, 1/2, 0, 1/2)) ← FusionTree{Irrep[SU₂]}((), 0, (), ()):
[:, :, 1, 1, 1, 1, 1] =
 2.4173545279602053e-19 - 0.008903336384610125im
* Data for fusiontree FusionTree{Irrep[SU₂]}((1, 1/2, 1, 1, 1/2, 1/2, 1/2), 0, (true, true, false, false, true, true, true), (1/2, 3/2, 3/2, 1, 1/2)) ← FusionTree{Irrep[SU₂]}((), 0, (), ()):
[:, :, 1, 1, 1, 1, 1] =
 -0.0002916452236111965 - 0.0016163951101132302im
* Data for fusiontree FusionTree{Irrep[SU₂]}((1/2, 0, 1, 1, 1/2, 1/2, 1/2), 0, (true, true, false, false, true, true, true), (1/2, 3/2, 1/2, 0, 1/2)) ← FusionTree{Irrep[SU₂]}((), 0, (), ())

In [22]:
init=Dict([("CTM", []), ("init_type", "PBC")]);
conv_check="singular_value";
@time CTM, AA_fused, U_L,U_D,U_R,U_U=CTMRG(A_fused,chi,conv_check,tol,init,CTM_ite_nums,CTM_trun_tol);


"initialize CTM"

"start CTM iterations:"

  0.586583 seconds (575.04 k allocations: 60.851 MiB)
corner 4:
[1.0, 0.4693875098099769, 0.4693875098099769, 0.4693875098099769, 0.4693875098099766, 0.1458788914340396, 0.14587889143403956, 0.1458788914340395, 0.12917641598709137, 0.09846223797234177, 0.09846223797234172, 0.09846223797234172, 0.05808750248342758, 0.058087502483427575, 0.05808750248342756, 0.05808750248342755, 0.018214713616901492, 0.018214713616901492, 0.018214713616901492, 0.018214713616901492, 0.01821471361690147, 0.01821471361690147, 0.01821471361690147, 0.01821471361690147, 4.456467588677625e-17, 3.5173034428270115e-17, 3.07111033975526e-17, 2.0162072164386798e-17, 1.6192548467893564e-17, 1.42723801041988e-17, 9.913076744599756e-18, 6.064690878216719e-18, 6.064690878216719e-18, 6.064690878216719e-18, 6.064690878216719e-18, 6.064690878216719e-18]
corner 1:
[1.0, 0.28097740011818917, 0.2809774001181891, 0.2809774001181891, 0.280977400118189, 0.06396928473252897, 0.06396928473252896, 0.06396928473252896, 0.0203729547

, 0.015699603875045993, 0.01569960387504599, 0.015699603875045987, 0.012514368303171132, 0.012514368303171128, 0.012507125256267407, 0.012507125256267406, 0.0060956643863997, 0.006095664386399698, 0.006095664386399697]
corner 3:
[1.0, 0.3686404099336842, 0.3686404099336841, 0.36863005111044644, 0.36863005111044633, 0.11443058037629485, 0.09821238642029827, 0.09821238642029827, 0.09821238642029827, 0.09090897128948151, 0.08393482020523593, 0.04108726427190297, 0.04108726427190296, 0.04107003180917885, 0.04107003180917884, 0.030108238234149626, 0.027168272761791273, 0.027168272761791266, 0.027168272761791255, 0.027044946984462397, 0.027044946984462394, 0.027016429521204716, 0.0270164295212047, 0.018681871533907633, 0.01868187153390763, 0.018681871533907622, 0.01565443504565223, 0.015654435045652226, 0.015654435045652226, 0.012407442827123341, 0.012407442827123341, 0.012402917406737055, 0.01240291740673699, 0.006086411484918117, 0.006086411484918113, 0.00608641148491811]
corner 2:
[1.0, 0

 0.5569984679912288, 0.5569984679912287, 0.3523196298173841, 0.3272234249008154, 0.29003391518673666, 0.23066020394258221, 0.23066020394258216, 0.2306602039425821, 0.22659815861982896, 0.22659815861982893, 0.2265967078752669, 0.22659670787526634, 0.1926992915226162, 0.15744586435554322, 0.15744586435554322, 0.15744586435554317, 0.15488081273375964, 0.15488081273375964, 0.154877881892183, 0.15487788189218296, 0.12974808334311722, 0.12974808334311722, 0.12974808334311716, 0.09723424831998267, 0.09723424831998262, 0.0972342483199826, 0.06195898194733438, 0.06195898194733438, 0.061957956593196806, 0.06195795659319652, 0.04870431989228536, 0.04870431989228536, 0.04870431989228535]
next iteration:
CTMRG iteration: 6, CTMRG err: 0.004927884141196094
corner 4:
[1.0, 0.6744945040761616, 0.6744945040761616, 0.6744935912982234, 0.6744935912982233, 0.45499582198023986, 0.4549958219802398, 0.45499582198023975, 0.3631209489293675, 0.3334484408562097, 0.2938469276383804, 0.2754412352246772, 0.2754412

Excessive output truncated after 524303 bytes.

, 0.2430097809937712, 0.2430097809937712, 0.22659326969747892, 0.2265932696974789, 0.22659325853585818, 0.22659325853585816, 0.21788168419410806, 0.21788168419410803, 0.21788168419410792, 0.2035365584573226, 0.18927929774867974, 0.18927929774867974, 0.1892792294528652, 0.1892792294528652, 0.1470729287646866, 0.14707292876468658, 0.14707292876468653, 0.14643761702091154, 0.14643761702091143, 0.14643761702091143]
corner 1:
[1.0, 0.37602158192351687, 0.37602158192351687, 0.3760215504658522, 0.3760215504658521, 0.12512152249788094, 0.10539150720616132, 0.1053915072061613, 0.10539150720616125, 0.10150928967059315, 0.09487086539771712, 0.05183272966372405, 0.051832729663724014, 0.05183268402530259, 0.051832684025302556, 0.0395103105182787, 0.03421832137252078, 0.034218321372520746, 0.03421828131616323, 0.03421828131616322, 0.03410254973102415, 0.03410254973102415, 0.03410254973102413, 0.0225172066830837, 0.0225172066830837, 0.0225172066830837, 0.02016207159898014, 0.02016207159898013, 0.0201

In [5]:

@time E_up, E_down=evaluate_ob(parameters, U_phy, A_unfused, A_fused, AA_fused, U_L,U_D,U_R,U_U, CTM, "E_triangle");
@time E_up_12, E_up_31, E_up_23, E_down_12, E_down_31, E_down_23=evaluate_ob(parameters, U_phy, A_unfused, A_fused, AA_fused, U_L,U_D,U_R,U_U, CTM, "E_bond");

display((E_up+E_down)/3)

display(space(CTM["Cset"][1]))
display(space(CTM["Cset"][2]))
display(space(CTM["Cset"][3]))
display(space(CTM["Cset"][4]))



# matwrite("Corners_julia"*"_D"*string(D)*"_chi"*string(chi)*".mat", Dict(
#     "C1" => svdvals(convert(Array, CTM["Cset"][1])),
#     "C2" => svdvals(convert(Array, CTM["Cset"][2])),
#     "C3" => svdvals(convert(Array, CTM["Cset"][3])),
#     "C4" => svdvals(convert(Array, CTM["Cset"][4]))
# ); compress = false)


UndefVarError: UndefVarError: AA_fused not defined

In [6]:
function build_double_layer(A,operator)
    #display(space(A))
    A=permute(A,(1,2,),(3,4,5));
    U_L=unitary(fuse(space(A, 1)' ⊗ space(A, 1)), space(A, 1)' ⊗ space(A, 1));
    U_D=unitary(fuse(space(A, 2)' ⊗ space(A, 2)), space(A, 2)' ⊗ space(A, 2));
    U_R=inv(U_L);
    U_U=inv(U_D);
    # display(space(U_L))
    # display(space(U_D))
    # display(space(U_R))
    # display(space(U_D))

    uM,sM,vM=tsvd(A);
    uM=uM*sM

    uM=permute(uM,(1,2,3,),())
    V=space(vM,1);
    U=unitary(fuse(V' ⊗ V), V' ⊗ V);
    @tensor double_LD[:]:=uM'[-1,-2,1]*U'[1,-3,-4];
    @tensor double_LD[:]:=double_LD[-1,-3,1,-5]*uM[-2,-4,1];

    vM=permute(vM,(1,2,3,4,),());
    if operator==[]
        @tensor double_RU[:]:=U[-1,-2,1]*vM[1,-3,-4,-5];
        @tensor double_RU[:]:=vM'[1,-2,-4,2]*double_RU[-1,1,-3,-5,2];
    else
        @tensor double_RU[:]:=U[-1,-2,1]*vM[1,-3,-4,-5];
        @tensor double_RU[:]:=vM'[3,-2,-4,1]*operator[2,1]*double_RU[-1,3,-3,-5,2];
    end
    #display(space(double_RU))

    double_LD=permute(double_LD,(1,2,),(3,4,5,));
    double_LD=U_L*double_LD;
    double_LD=permute(double_LD,(2,3,),(1,4,));
    double_LD=U_D*double_LD;
    double_LD=permute(double_LD,(2,1,),(3,));
    #display(space(double_LD))
    double_RU=permute(double_RU,(1,4,5,),(2,3,));
    double_RU=double_RU*U_R;
    double_RU=permute(double_RU,(1,4,),(2,3,));
    double_RU=double_RU*U_U;
    double_LD=permute(double_LD,(1,2,),(3,));
    double_RU=permute(double_RU,(1,),(2,3,));
    AA_fused=double_LD*double_RU;

    return AA_fused, U_L,U_D,U_R,U_U
end


build_double_layer (generic function with 1 method)

In [7]:
function fuse_CTM_legs(CTM,U_L,U_D,U_R,U_U)
    #fuse CTM legs
    Tset=CTM["Tset"];

    #T4
    T4=permute(Tset[4],(1,4,),(2,3,));
    T4=T4*U_R;
    T4=permute(T4,(1,3,2,),());
    Tset[4]=T4
    #display(space(T4))

    #T3
    T3=permute(Tset[3],(1,4,),(2,3,));
    T3=T3*U_U;
    T3=permute(T3,(1,3,2,),());
    Tset[3]=T3
    #display(space(T3))

    #T2
    T2=permute(Tset[2],(2,3,),(1,4,));
    T2=U_L*T2;
    T2=permute(T2,(2,1,3,),());
    Tset[2]=T2
    #display(space(T2))

    #T1
    T1=permute(Tset[1],(2,3,),(1,4,));
    T1=U_D*T1;
    T1=permute(T1,(2,1,3,),());
    Tset[1]=T1
    #display(space(T1))

    CTM["Tset"]=Tset;
    return CTM
end


fuse_CTM_legs (generic function with 1 method)

In [20]:
function CTMRG(A,chi,conv_check,tol,init,CTM_ite_nums, CTM_trun_tol)

    #Ref: PHYSICAL REVIEW B 98, 235148 (2018)
    #initial corner transfer matrix
        #initial corner transfer matrix
    if isempty(init["CTM"])
        CTM, AA_fused, U_L,U_D,U_R,U_U=init_CTM(chi,A,init["init_type"]);
    else
        CTM=init;
    end

    Cset=CTM["Cset"];
    Tset=CTM["Tset"];
    conv_check="singular_value"

    ss_old=ones(chi)*2;
    d=2;
    rho_old=Matrix(I,d^3,d^3);

    #Iteration

    print_corner=true;
    if print_corner
        println("corner 4:")
        C4_spec=svdvals(convert(Array,Cset[4]));
        C4_spec=C4_spec/C4_spec[1];
        println(C4_spec);
        println("corner 1:")
        C1_spec=svdvals(convert(Array,Cset[1]));
        C1_spec=C1_spec/C1_spec[1];
        println(C1_spec);
        println("corner 3:")
        C3_spec=svdvals(convert(Array,Cset[3]));
        C3_spec=C3_spec/C3_spec[1];
        println(C3_spec);
        println("corner 2:")
        C2_spec=svdvals(convert(Array,Cset[2]));
        C2_spec=C2_spec/C2_spec[1];
        println(C2_spec);
    end
    println("CTM init finished")



    display("start CTM iterations:")
    for ci=1:CTM_ite_nums
        #direction_order=[1,2,3,4];
        direction_order=[4,1,2,3];
        for direction in direction_order
            Cset,Tset=CTM_ite(Cset, Tset, AA_fused, chi, direction,CTM_trun_tol);
        end

        print_corner=true;
        if print_corner
            println("corner 4:")
            C4_spec=svdvals(convert(Array,Cset[4]));
            C4_spec=C4_spec/C4_spec[1];
            println(C4_spec);
            println("corner 1:")
            C1_spec=svdvals(convert(Array,Cset[1]));
            C1_spec=C1_spec/C1_spec[1];
            println(C1_spec);
            println("corner 3:")
            C3_spec=svdvals(convert(Array,Cset[3]));
            C3_spec=C3_spec/C3_spec[1];
            println(C3_spec);
            println("corner 2:")
            C2_spec=svdvals(convert(Array,Cset[2]));
            C2_spec=C2_spec/C2_spec[1];
            println(C2_spec);
        end
        println("next iteration:")


        if conv_check=="singular_value" #check convergence of singular value
            _,ss_new,_=svd(permute(Cset[1],(1,),(2,)));
            ss_new=convert(Array,ss_new);
            ss_new=sort(diag(ss_new), rev=true);
            ss_old=ss_old/ss_old[1];
            ss_new=ss_new/ss_new[1];
            #display(ss_new)
            if length(ss_old)>length(ss_new)
                dss=copy(ss_old);
                siz=length(ss_new)
            elseif length(ss_old)<=length(ss_new)
                dss=copy(ss_new);
                siz=length(ss_old)
            end
            dss[1:siz]=ss_old[1:siz]-ss_new[1:siz]
            er=norm(dss);
            println("CTMRG iteration: "*string(ci)*", CTMRG err: "*string(er));
            if er<tol
                break;
            end
            ss_old=ss_new;
        elseif conv_check=="density_matrix" #check reduced density matrix

            # ob_opts.SiteNumber=1;
            # CTM_tem.Cset=Cset;
            # CTM_tem.Tset=Tset;
            # rho_new=ob_CTMRG(CTM_tem,A,ob_opts).A;
            # er=sum(sum((abs(rho_old-rho_new))));
            # disp(['CTMRG iteration: ',num2str(ci),' CTMRG err: ',num2str(er)]);
            # if er<tol
            #     break;
            # end
            # rho_old=rho_new;
        end

        # if ci==CTM_ite_nums
        #     display(er)
        #     warn("CTMRG does not converge: " * string(er));
        # end
    end

    CTM["Cset"]=Cset;
    CTM["Tset"]=Tset;
    return CTM, AA_fused, U_L,U_D,U_R,U_U

end


CTMRG (generic function with 1 method)

In [9]:
function CTM_ite(Cset, Tset, AA_fused, chi, direction, trun_tol)

    AA=permute(AA_fused, (mod1(2-direction,4),mod1(3-direction,4),mod1(4-direction,4),mod1(1-direction,4),),());

    @tensor MMup[:]:=Cset[mod1(direction,4)][1,2]*Tset[mod1(direction,4)][2,3,-3]*Tset[mod1(direction-1,4)][-1,4,1]*AA[4,-2,-4,3];
    @tensor MMlow[:]:=Tset[mod1(direction-1,4)][1,3,-1]*AA[3,4,-4,-2]*Cset[mod1(direction-1,4)][2,1]*Tset[mod1(direction-2,4)][-3,4,2];

    @tensor MMup_reflect[:]:=Tset[mod1(direction,4)][-1,3,1]* Cset[mod1(direction+1,4)][1,2]* AA[-2,-4,4,3]* Tset[mod1(direction+1,4)][2,4,-3];
    #@tensor MMlow_reflect[:]:=AA[-2,4,3,-4]*Tset[mod1(direction+1,4)][-3,3,1]*Tset[mod1(direction-2,4)][2,4,-1]*Cset[mod1(direction-2,4)][1,2]; #this is slow compared to other coners, I don't know why
    @tensor MMlow_reflect[:]:=Tset[mod1(direction+1,4)][-4,-3,2]*Tset[mod1(direction-2,4)][1,-2,-1]*Cset[mod1(direction-2,4)][2,1];
    @tensor MMlow_reflect[:]:=MMlow_reflect[-1,1,2,-3]*AA[-2,1,2,-4];

    MMup=permute(MMup,(1,2,),(3,4,))

    # _,ss,_=tsvd(MMup)
    # display(convert(Array,ss))

    MMlow=permute(MMlow,(1,2,),(3,4,))
    MMup_reflect=permute(MMup_reflect,(1,2,),(3,4,))
    MMlow_reflect=permute(MMlow_reflect,(1,2,),(3,4,))

    qr_decomp=false;
    if qr_decomp

        _, RMup=leftorth(permute(MMup*MMup_reflect,(3,4,),(1,2,)));
        _, RMlow=leftorth(permute(MMlow*MMlow_reflect,(3,4,),(1,2,)));


        RMlow=permute(RMlow,(2,3,),(1,));
        M=RMup*RMlow;



        uM,sM,vM = tsvd(M; trunc=truncdim(chi));
        #uM,sM,vM = tsvd(M; trunc=trunbelow(1e-34)); #this looks to truncate absolute value, not relative value
        #display(convert(Array,sM))
        #display(convert(Array,inv(sM)))
        sM=sM/norm(sM)
        sM_inv=inv(sM);
        sM_dense=convert(Array,sM)
        #display(sM_dense)

        for c1=1:size(sM_dense,1)
            if sM_dense[c1,c1]<trun_tol
                sM_dense[c1,c1]=0;
            end
        end
        #display(sM_dense)
        #display(pinv.(sM_dense))

        #display(sM_inv)
        #display(convert(Array,sM_inv))
        sM_inv_sqrt=sqrt.(convert(Array,sM_inv))
        #display(space(sM_inv))
        #display(sM_inv_sqrt)
        sM_inv_sqrt=TensorMap(pinv.(sqrt.(sM_dense)),codomain(sM_inv)←domain(sM_inv))

        PM_inv=RMlow*vM'*sM_inv_sqrt;
        PM=sM_inv_sqrt*uM'*RMup;
        PM=permute(PM,(2,3,),(1,));

        @tensor M5tem[:]:=Tset[mod1(direction-1,4)][4,3,1]*AA[3,5,-2,2]* PM_inv[4,5,-1]* PM[1,2,-3];
        @tensor M1tem[:]:=Cset[mod1(direction,4)][1,2]*Tset[mod1(direction,4)][2,3,-2]*PM_inv[1,3,-1];
        @tensor M7tem[:]:=Cset[mod1(direction-1,4)][1,2]*Tset[mod1(direction-2,4)][-1,3,1]* PM[2,3,-2];
    else
        RMup=permute(MMup*MMup_reflect,(3,4,),(1,2,));
        RMlow=MMlow*MMlow_reflect;

        M=RMup*RMlow;

        uM,sM,vM = tsvd(M; trunc=truncdim(chi));
        #uM,sM,vM = tsvd(M; trunc=trunbelow(1e-34)); #this looks to truncate absolute value, not relative value
        #display(convert(Array,sM))
        #display(convert(Array,inv(sM)))
        sM=sM/norm(sM)
        sM_inv=inv(sM);
        sM_dense=convert(Array,sM)
        #display(sM_dense)

        for c1=1:size(sM_dense,1)
            if sM_dense[c1,c1]<trun_tol
                sM_dense[c1,c1]=0;
            end
        end
        #display(sM_dense)
        #display(pinv.(sM_dense))

        #display(sM_inv)
        #display(convert(Array,sM_inv))
        sM_inv_sqrt=sqrt.(convert(Array,sM_inv))
        #display(space(sM_inv))
        #display(sM_inv_sqrt)
        sM_inv_sqrt=TensorMap(pinv.(sqrt.(sM_dense)),codomain(sM_inv)←domain(sM_inv))

        PM_inv=RMlow*vM'*sM_inv_sqrt;
        PM=sM_inv_sqrt*uM'*RMup;
        PM=permute(PM,(2,3,),(1,));

        @tensor M5tem[:]:=Tset[mod1(direction-1,4)][4,3,1]*AA[3,5,-2,2]* PM_inv[4,5,-1]* PM[1,2,-3];
        @tensor M1tem[:]:=Cset[mod1(direction,4)][1,2]*Tset[mod1(direction,4)][2,3,-2]*PM_inv[1,3,-1];
        @tensor M7tem[:]:=Cset[mod1(direction-1,4)][1,2]*Tset[mod1(direction-2,4)][-1,3,1]* PM[2,3,-2];
    end

    Cset[mod1(direction,4)]=M1tem/norm(M1tem);
    Tset[mod1(direction-1,4)]=M5tem/norm(M5tem);
    Cset[mod1(direction-1,4)]=M7tem/norm(M7tem);
    return Cset,Tset
end


CTM_ite (generic function with 1 method)

In [10]:
function init_CTM(chi,A,type)

    display("initialize CTM")
    #numind(A)
    #numin(A)
    #numout(A)
    CTM=[];
    Cset=Vector(undef,4);
    Tset=Vector(undef,4);
    #space(A,1)
    @time if type=="PBC"
        for direction=1:4
            inds=(mod1(2-direction,4),mod1(3-direction,4),mod1(4-direction,4),mod1(1-direction,4),5);
            A_rotate=permute(A,inds);
            Ap_rotate=A_rotate';

            @tensor M[:]:=Ap_rotate[1,-1,-3,2,3]*A_rotate[1,-2,-4,2,3];
            Cset[direction]=M;
            @tensor M[:]:=Ap_rotate[-1,-3,-5,1,2]*A_rotate[-2,-4,-6,1,2];
            Tset[direction]=M;
        end

        #fuse legs
        ul_set=Vector(undef,4);
        ur_set=Vector(undef,4);
        for direction=1:2
            ul_set[direction]=unitary(fuse(space(Cset[direction], 3) ⊗ space(Cset[direction], 4)), space(Cset[direction], 3) ⊗ space(Cset[direction], 4));
            ur_set[direction]=unitary(fuse(space(Tset[direction], 5) ⊗ space(Tset[direction], 6)), space(Tset[direction], 5) ⊗ space(Tset[direction], 6));
        end
        for direction=3:4
            ul_set[direction]=unitary(fuse(space(Cset[direction], 3) ⊗ space(Cset[direction], 4))', space(Cset[direction], 3) ⊗ space(Cset[direction], 4));
            ur_set[direction]=unitary(fuse(space(Tset[direction], 5) ⊗ space(Tset[direction], 6))', space(Tset[direction], 5) ⊗ space(Tset[direction], 6));
        end
        for direction=1:4
            C=Cset[direction];
            ul=ur_set[mod1(direction-1,4)];
            ur=ul_set[direction];
            ulp=permute(ul',(3,),(1,2,));
            urp=permute(ur',(3,),(1,2,));
            #@tensor Cnew[(-1);(-2)]:=ulp[-1,1,2]*C[1,2,3,4]*ur[-2,3,4]
            @tensor Cnew[:]:=ulp[-1,1,2]*C[1,2,3,4]*ur[-2,3,4];#put all indices in tone side so that its adjoint has the same index order
            Cset[direction]=Cnew;

            T=Tset[direction];
            ul=ul_set[direction];
            ur=ur_set[direction];
            ulp=permute(ul',(3,),(1,2,));
            urp=permute(ur',(3,),(1,2,));
            #@tensor Tnew[(-1);(-2,-3,-4)]:=ulp[-1,1,2]*T[1,2,-2,-3,3,4]*ur[-4,3,4]
            @tensor Tnew[:]:=ulp[-1,1,2]*T[1,2,-2,-3,3,4]*ur[-4,3,4];#put all indices in tone side so that its adjoint has the same index order
            Tset[direction]=Tnew;
        end
    elseif type=="random"
    end
    CTM=Dict([("Cset", Cset), ("Tset", Tset)]);

    AA_fused, U_L,U_D,U_R,U_U=build_double_layer(A,[]);
    CTM=fuse_CTM_legs(CTM,U_L,U_D,U_R,U_U);

    return CTM, AA_fused, U_L,U_D,U_R,U_U


    # #save initial CTM to compare with other codes
    # @time CTM=init_CTM(10,PEPS_tensor,"PBC");
    # matwrite("matfile.mat", Dict(
    # 	"C1" => convert(Array,CTM["Cset"][1]),
    # 	"C2" => convert(Array,CTM["Cset"][2]),
    #     "C3" => convert(Array,CTM["Cset"][3]),
    #     "C4" => convert(Array,CTM["Cset"][4]),
    #     "T1" => convert(Array,CTM["Tset"][1]),
    #     "T2" => convert(Array,CTM["Tset"][2]),
    #     "T3" => convert(Array,CTM["Tset"][3]),
    #     "T4" => convert(Array,CTM["Tset"][4])
    # ); compress = false)

    end;
